In [1]:
import os
import wave
import time
import pickle
import pyaudio
import warnings
import numpy as np
from sklearn import preprocessing
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QComboBox, QLineEdit, QPushButton, QMessageBox
from PyQt5.QtCore import QTimer

warnings.filterwarnings("ignore")




In [2]:
def calculate_delta(array):
   
    rows,cols = array.shape
    print('rows:'+str(rows))
    print('cols:'+str(cols))
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
              first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

In [3]:
def extract_features(audio,rate):
#     Mel-frequency cepstral coefficients
    mfcc_feature = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)
    print('mfcc_features:'+str(mfcc_feature))
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature,delta)) 
    return combined

In [4]:
def test_model():
    print("Running test_model()")
    source   = "testing_set"
    modelpath = "trained_models"
    test_file = "testing_set_addition.txt"
    file_paths = open(test_file,'r')

    gmm_files = [os.path.join(modelpath,fname) for fname in
                  os.listdir(modelpath) if fname.endswith('.gmm')]

    #Load the Gaussian gender Models
    models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
    speakers   = [fname.split("\\")[-1].split(".gmm")[0] for fname 
                  in gmm_files]

    # Read the test directory and get the list of test audio files 
    winner=''
    for path in file_paths:   

        path = path.strip()   
        print('path:'+str(path))
        sr,audio = read(source + "\\"+ path)
        vector   = extract_features(audio,sr)

        log_likelihood = np.zeros(len(models)) 

        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()
            print('log_liklihood[i]:'+str(log_likelihood[i]))

        winner = np.argmax(log_likelihood)
        print("\tdetected as - ", speakers[winner])
        time.sleep(1.0)
        return speakers[winner]
        
    
    

  


In [5]:
class VoiceRecognitionAttendanceSystem(QWidget):
    def __init__(self):
        super().__init__()
        
        # set window title and size
        self.setWindowTitle("Voice Recognition Attendance System")
        self.setGeometry(600, 300, 600, 400)
        
        # create GUI elements
        self.course_label = QLabel("Select Course Code:", self)
        self.course_label.move(200, 50)
        
        self.course_combo = QComboBox(self)
        self.course_combo.addItems(["AI-420", "MS-434","AI-407L"])
        self.course_combo.move(330, 50)
        
        self.reg_label = QLabel("Enter Your Name:", self)
        self.reg_label.move(170, 90)
        
        self.reg_input = QLineEdit(self)
        self.reg_input.move(300, 90)
        
        self.attendance_button = QPushButton("Mark Attendance", self)
        self.attendance_button.move(245, 150)
        self.attendance_button.clicked.connect(self.record_audio_test)
        
        self.exit_button = QPushButton("Exit", self)
        self.exit_button.move(250, 210)
        self.exit_button.clicked.connect(self.close)
        
        # initialize timer for countdown
        self.timer = QTimer(self)
        self.timer.setInterval(1000)  # 1 second
        
        
        
    def record_audio_test(self):
        # disable the button to prevent multiple clicks
        self.attendance_button.setEnabled(False)
        
#        create message box to show "Recording..." message
        recording_box = QMessageBox(self)
        recording_box.setWindowTitle("Recording...")
        recording_box.setText("Please say your name")
        recording_box.show()
        
#         Name =(input("Please Enter Your Name:"))
        Name=self.reg_input.text()
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 44100
        CHUNK = 512
        RECORD_SECONDS = 5
        device_index = 1
        audio = pyaudio.PyAudio()
        print("----------------------record device list---------------------")
        info = audio.get_host_api_info_by_index(0)
        numdevices = info.get('deviceCount')
        for i in range(0, numdevices):
                if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
                    print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))
        print("-------------------------------------------------------------")
        index = 1
        print("recording via index ",index)
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,input_device_index = index,
                        frames_per_buffer=CHUNK)
        print ("recording started")
        Recordframes = []
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            Recordframes.append(data)
        print ("recording stopped")
        stream.stop_stream()
        stream.close()
        audio.terminate()
        OUTPUT_FILENAME=Name+"-testsample"+".wav"
        WAVE_OUTPUT_FILENAME=os.path.join("testing_set",OUTPUT_FILENAME)
        trainedfilelist = open("testing_set_addition.txt", 'a')
        trainedfilelist.write(OUTPUT_FILENAME+"\n")
        waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(Recordframes))
        waveFile.close()
        
#     OUTPUT_FILENAME=Name+"-testsample"+".wav"
#     WAVE_OUTPUT_FILENAME=os.path.join("testing_set",OUTPUT_FILENAME)
#     trainedfilelist = open("testing_set_addition.txt", 'a')
#     trainedfilelist.write(OUTPUT_FILENAME+"\n")
#     waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
#     waveFile.setnchannels(CHANNELS)
#     waveFile.setsampwidth(audio.get_sample_size(FORMAT))
#     waveFile.setframerate(RATE)
#     waveFile.writeframes(b''.join(Recordframes))
#     waveFile.close()

        
        recording_box.close()
        speaker= test_model()
        QMessageBox.information(self, "Voice Recognition Attendance System", "Attendance for "+str(speaker)+" Marked!")
        self.attendance_button.setEnabled(True)

    
if __name__ == '__main__':
    # create the application and show the GUI
    app = QApplication(sys.argv)
    window = VoiceRecognitionAttendanceSystem()
    window.show()
    sys.exit(app.exec_())

# while True:
#     choice=int(input("\n 1.Record audio for training \n 2.Train Model \n 3.Record audio for testing \n 4.Test Model\n"))
#     if(choice==1):
#         record_audio_train()
#     elif(choice==2):
#         train_model()
#     elif(choice==3):
#         record_audio_test()
#     elif(choice==4):
#         test_model()
#     if(choice>4):
#         exit()

SystemExit: 0